In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Random tests with pythons library

In [19]:
"""
import pandas as pd
import numpy as np
"""

'\nimport pandas as pd\nimport numpy as np\n'

In [20]:
"""
data = pd.read_csv(
    "https://www.data.gouv.fr/fr/datasets/r/3004168d-bec4-44d9-a781-ef16f41856a2",
    sep="|",
    decimal=","
    # dtype={'Valeur fonciere': np.float64}
)
"""

'\ndata = pd.read_csv(\n    "https://www.data.gouv.fr/fr/datasets/r/3004168d-bec4-44d9-a781-ef16f41856a2",\n    sep="|",\n    decimal=","\n    # dtype={\'Valeur fonciere\': np.float64}\n)\n'

In [21]:
"""
print(data["Valeur fonciere"][:10])
print(data["Surface reelle bati"][:10])
"""

'\nprint(data["Valeur fonciere"][:10])\nprint(data["Surface reelle bati"][:10])\n'

In [22]:
"""
data

#print(data[:1])
print(data
      .filter(items=['Valeur fonciere', 'Surface reelle bati', 'Type local'])
      .groupby(by="Type local")
      .mean()
)
"""

'\ndata\n\n#print(data[:1])\nprint(data\n      .filter(items=[\'Valeur fonciere\', \'Surface reelle bati\', \'Type local\'])\n      .groupby(by="Type local")\n      .mean()\n)\n'

In [23]:
"""
refinedData = data[(data["Type local"] == "Appartement") | (data["Type local"] == "Maison")]
"""

'\nrefinedData = data[(data["Type local"] == "Appartement") | (data["Type local"] == "Maison")]\n'

In [24]:
"""
print(refinedData[:10]
      .filter(items=['Valeur fonciere', 'Surface reelle bati', 'Type local'])
      .groupby(by="Type local")
      .mean()
)
"""

'\nprint(refinedData[:10]\n      .filter(items=[\'Valeur fonciere\', \'Surface reelle bati\', \'Type local\'])\n      .groupby(by="Type local")\n      .mean()\n)\n'

In [25]:
"""
import plotly.express as px

fig = px.scatter(
    x=refinedData["Surface reelle bati"][:300],
    y=refinedData["Valeur fonciere"][:300]
)

fig.show()
"""

'\nimport plotly.express as px\n\nfig = px.scatter(\n    x=refinedData["Surface reelle bati"][:300],\n    y=refinedData["Valeur fonciere"][:300]\n)\n\nfig.show()\n'

In [26]:
"""
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=refinedData["Surface reelle bati"][:300],
    y=refinedData["Valeur fonciere"][:300],
    mode='markers'
))

def f(x):
    return 1600 * x + 10000

x = np.linspace(0, 500, 300)
y = np.vectorize(f)(x)

print(x[:3])
print(y[:3])

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='lines'
))

fig.show()
"""

'\nimport plotly.graph_objects as go\n\nfig = go.Figure()\n\nfig.add_trace(go.Scatter(\n    x=refinedData["Surface reelle bati"][:300],\n    y=refinedData["Valeur fonciere"][:300],\n    mode=\'markers\'\n))\n\ndef f(x):\n    return 1600 * x + 10000\n\nx = np.linspace(0, 500, 300)\ny = np.vectorize(f)(x)\n\nprint(x[:3])\nprint(y[:3])\n\nfig.add_trace(go.Scatter(\n    x=x,\n    y=y,\n    mode=\'lines\'\n))\n\nfig.show()\n'

In [27]:
"""
trainingSet = refinedData.filter(items=[
    'Valeur fonciere',
    'Surface reelle bati'
])
"""

"\ntrainingSet = refinedData.filter(items=[\n    'Valeur fonciere',\n    'Surface reelle bati'\n])\n"

# Defining some helpers function to plot things in Graph.py

In [28]:
from Graph import Graph

# Step 1: Simple linear regression (univariate linear regression)


## Hypothesis

The hypothesis is the function that for each $x$ in the training set will give us the best estimation of each $y$.

Simply put, the hypothesis is the function that "solve" the problem at hands.

In the case of a univariate (one variable) linear regression the hypothesis function is a simple linear function:

$h_\theta(x) = \theta_0 + \theta_1x$

Our goal then is to find $\theta_0$ and $\theta_1$ so that $h_\theta(x)$ is close to $y$ for each of our training examples $(x, y)$

In [29]:
class Hypothesis:
    theta0 = None
    theta1 = None

    def __init__(self, theta0, theta1):
        self.theta0 = theta0
        self.theta1 = theta1

    def compute(self, x):
        return self.theta0 + (self.theta1 * x)

hypo1 = Hypothesis(10, 2)
hypo2 = Hypothesis(100, 0)

graph = Graph()
graph.plotFunction(hypo1.compute, "hypo1")
graph.plotFunction(hypo2.compute, "hypo2")
graph.render()

## Cost function

In order to find the best values for $\theta_0$ and $\theta_1$ we define the cost function as follow (m being the number of training example in our dataset):

$ J_{(\theta_0, \theta_1)} = \frac{1}{2m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^i)^2 $

And the problem become minimizing this function over $\theta_0$ and $\theta_1$.

Let's unpack the formula:

- $ h_\theta(x^{(i)}) $

Is our hypothesis function as defined last chapter applied to the $i^{th}$ example of our training data.

- $ (h_\theta(x^{(i)}) - y^i)^2 $

Is the "squared error function".

It represents the "error", how much apart from the right result $y^i$ the result of our hypothesis $h_\theta(x^{(i)})$ is.

And we take the square of that error supposedly because the square function accentuate the difference between large and small error (an "error" of 5 becomes 25 while an error of 2 become 4 making the error of 5 appear even worse)

The "squared error function" is one example of an error function, there are others. But the squared error function is one that is generally quite good.

- $ \frac{1}{2m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^i)^2 $

This $\frac{1}{2m} \sum_{i=1}^m$ part is the notation to take the average. (Sum of each elements divided by the number of elements)

So the formulas is taking the average of the error of our hypothesis over our entire dataset.

In [30]:
class CostFunction:
    dataset = None
    hypothesis = None

    def __init__(self, dataset, hypothesis = None):
        self.dataset = dataset
        self.hypothesis = hypothesis

    def squaredError(self, theta0 = None, theta1 = None):
        # If theta0 and theta1 are given, we ignore the registered
        # hypothesis and create a new one with the given parameters
        if theta0 != None and theta1 != None:
            hypothesis = Hypothesis(theta0, theta1)
        else:
            hypothesis = self.hypothesis

        m = len(self.dataset[0])

        squaredErrorSum = 0

        for i in range(1, m):
            hx = hypothesis.compute(self.dataset[0][i])
            y = self.dataset[1][i]
            squaredErrorSum += (hx - y) ** 2

        return (1 / (2 * m)) * squaredErrorSum


dataset = [
    [ 0,  1,  2,  3],
    [10, 14, 18, 22]
]

hypo1 = Hypothesis(10, 4)
cf1 = CostFunction(dataset, hypo1)

hypo2 = Hypothesis(20, 1)
cf2 = CostFunction(dataset,hypo2)

# Graph that plots the dataset and the 2 hypothesis to show that hypo1 is the right hypothesis
graph = Graph()
graph.plot2DArray(dataset, "dataset")
graph.plotFunction(hypo1.compute, "hypo1", 0, 4, 4)
graph.plotFunction(hypo2.compute, "hypo2", 0, 4, 4)
graph.render()

print(
    "Right hypothesis, so squared error should be: ",
    cf1.squaredError(),
    " == 0"
)

print(
    "Wrong hypothesis, so squared error should be: ",
    cf2.squaredError(),
    " != 0"
)

Right hypothesis, so squared error should be:  0.0  == 0
Wrong hypothesis, so squared error should be:  8.25  != 0


### Side note: 3D plot of the cost function

We can plot the cost function in 3D where $x$ and $y$ are the $\theta_0$ and $\theta_1$ parameters and z is the value of our cost function for those parameters.

This gives us a bow shaped surface in which our goal is to find the minima.

In [31]:
contour = Graph()
contour.plot3DArray([[10], [4], [0]], "minima")
contour.plot3DSurface(
    CostFunction(dataset).squaredError,
    -11, 21, 31
)
contour.render()

### Side note: Contour line

The contour line is another way to plot our cost function where we are able to plot it in 2D which is easier to visualize.

The resulting graph shows curve of "level" of errors, and the minima for the cost function is at the middle of those curves.

In [49]:
contour = Graph()
contour.plot2DArray([[10], [4]], "minima")
contour.plotContourLine(
    CostFunction(dataset).squaredError,
    0, 14, 40
)
contour.render()

## Gradient descent

repeat until convergence {

$ \theta_j := \theta_j - \alpha \frac{\partial}{\partial \theta_j} J(\theta_o, \theta_1) $

}

# Questions:

### 1) In $ \frac{1}{2m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^i)^2 $ why $ \frac{1}{2m} $ and not simply $ \frac{1}{m} $ like a regular average ?

A youtube comments says: we are dividing by m to get the mean (average) and 2 is for simplification of the derivative (a^2 =2a)

What does that mean ? Is $ \frac{1}{2m} $ simpler to derive than $ \frac{1}{m} $ ?

### 2) How to figure out a good range for a contour plot ?

I can't find a good range of value for which my contour plot looks nice and pretty
